In [2]:
%load_ext watermark
%watermark

2017-12-04T15:11:19+01:00

CPython 3.6.1
IPython 6.2.1

compiler   : GCC 4.8.2 20140120 (Red Hat 4.8.2-15)
system     : Linux
release    : 4.10.0-40-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 8
interpreter: 64bit


In [1]:
from IPython.display import Image
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

matplotlib.rcParams['figure.figsize'] = [10, 10]

Para garantizar la reproducibilidad (esto es, que al ejecutar este notebook otra vez los resultados sean idénticos) vamos a fijar la semilla que usa scikit learn para hacer particiones aleatorias. Para esto tenemos que fijar la semilla (seed) cada vez que llamemos a la aplicacion.

In [2]:
 np.random.seed(42)

# Selección de variables

En esta sección vamos a ver las distintas técnicas de selección de variables, que son las técnicas que nos permiten filtrar las variables que tenemos y elegir aquellas que realmente están ayudando al modelo.

¿Por qué querriamos eliminar variables? Generalmente, cuanta más información tenemos sobre cada observación mejor será la calidad de nuestras predicciones (cuanto más sepamos, más facil será diferenciar unos casos de otros). No obstante, esto no tiene por que ser así en todas las situaciones, hay veces que ciertas variables pueden ser contraproducentes y hacer que nuestros modelos funcionen peor, por ejemplo debido a errores de medición.

Seleccionar variables tiene varias ventajas. En primer lugar, como hemos dicho, puede mejorar las puntuaciones de los modelos (eliminando parte del ruido en los datos), por otra parte, reducir el número de variables independientes reduce la complejidad del modelo, reduciendo así el efecto de sobreajuste del modelo. Además, tener menos variables independientes y modelos más simples significa que nuestros modelos son más rápidos de entrenar, y podemos entrenarlos con más datos en las mismas máquinas.

## Cargamos los datos

Para esta sección vamos a usar el [Ames Housing dataset](https://ww2.amstat.org/publications/jse/v19n3/decock/DataDocumentation.txt) que es una versión más moderna y con muchas más variables que el Boston Housing Dataset. Tiene información sobre casas en la ciudad de Ames, en Iowa, y su precio de venta. Este [link](https://ww2.amstat.org/publications/jse/v19n3/decock/DataDocumentation.txt) contiene la descripción de las variables en inglés.

In [3]:
ames = pd.read_csv("data/ames.csv").drop("id_parcela", axis=1) # no necesitamos el identificador de la parcela

In [4]:
ames.shape

(2930, 80)

In [5]:
ames.head()

,tipo_construccion,tipo_zona,perimetro_conectado_calle,area_parcela,tipo_acceso,tipo_acceso_trasero,forma_parcela,LandContour,tipo_instalaciones,config_parcela,...,area_piscina,calidad_piscina,calidad_valla,atributo_miscelaneo,valor_atributo_miscelaneo,mes_venta,ano_venta,tipo_venta,condicion_venta,precio_venta
0,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [7]:
ames.shape

(2930, 80)

In [6]:
ames.sample(1).T

,1357
tipo_construccion,70
tipo_zona,RM
perimetro_conectado_calle,NaN
area_parcela,5100
tipo_acceso,Pave
tipo_acceso_trasero,Grvl
forma_parcela,Reg
LandContour,Lvl
tipo_instalaciones,AllPub
config_parcela,Inside


In [8]:
ames.precio_venta.describe()

count      2930.000000
mean     180796.060068
std       79886.692357
min       12789.000000
25%      129500.000000
50%      160000.000000
75%      213500.000000
max      755000.000000
Name: precio_venta, dtype: float64

In [9]:
ames.dtypes

tipo_construccion                int64
tipo_zona                       object
perimetro_conectado_calle      float64
area_parcela                     int64
tipo_acceso                     object
tipo_acceso_trasero             object
forma_parcela                   object
LandContour                     object
tipo_instalaciones              object
config_parcela                  object
pendiente_parcela               object
barrio                          object
cercania_carretera1             object
cercania_carretera2             object
tipo_edificio                   object
tipo_casa                       object
calidad_general                  int64
condicion_general                int64
fecha_construccion               int64
ano_remodelado                   int64
tipo_tejado                     object
material_tejado                 object
material_exterior1              object
material_exterior2              object
tipo_revestimiento              object
area_revestimiento       

## Procesado de datos

Para poder utilizar este dataset tenemos que transformar las variables categóricas en numéricas. En vez de crear nuestro propio `ColumnSelector` vamos a usar el de la libreria auxiliar `mlxtend`

In [10]:
variables_independientes = ames.drop("precio_venta", axis=1).columns
variable_objetivo = ["precio_venta"]

In [11]:
datos_numericos = ames[variables_independientes].select_dtypes([int, float])
col_no_numericas = ames[variables_independientes].select_dtypes([object]).columns

col_numericas = datos_numericos.columns

# para crear este diccionario he ido a la descripcion del dataset
dict_var_ordinales = {
    "calidad_cocinas": ["Po", "Fa", "TA", "Gd", "Ex"],
    "funcionalidad":["Sal", "Sev", "Maj2", "Maj1", "Min2", "Min1","Typ"],
    "calidad_chimeneas":["NA","Po","Fa","TA","Gd","Ex"],
    "acabado_garaje":["NA","Unf","RFn","Fin"],
    "calidad_garaje":["NA","Po","Fa","TA","Gd","Ex"],
    "condicion_garaje":["NA","Po","Fa","TA","Gd","Ex"],
    "acceso_garaje_pavimentado":["N", "P", "Y"],
    "calidad_piscina":["NA","Fa","TA","Gd","Ex"],
    "calidad_valla":["NA","MnWw","GdWo","MnPrv","GdPrv"],
    "forma_parcela":["IR3", "IR2", "IR1","Reg"],
    "tipo_instalaciones":["ELO","NoSeWa","NoSewr","AllPub"],
    "pendiente_parcela":["Sev", "Mod", "Gtl"],
    "calidad_material_exterior":["Po","Fa","TA","Gd","Ex"],
    "condicion_material_exterior":["Po","Fa","TA","Gd","Ex"],
    "altura_sotano":["NA","Po","Fa","TA","Gd","Ex"],
    "condicion_sotano":["NA","Po","Fa","TA","Gd","Ex"],
    "sotano_exterior":["NA","No","Mn","Av","Gd"],
    "calidad_sotano_habitable1":["NA", "Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"],
    "calidad_sotano_habitable2":["NA", "Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"],
     "calidad_calefaccion":["Po","Fa","TA","Gd","Ex"],
}
col_ordinales = list(dict_var_ordinales.keys())
datos_ordinales = ames[col_ordinales]
col_categoricas = list(set(col_no_numericas) - set(col_ordinales))
datos_categoricos = ames[col_categoricas]


In [12]:
datos_numericos.head()

,tipo_construccion,perimetro_conectado_calle,area_parcela,calidad_general,condicion_general,fecha_construccion,ano_remodelado,area_revestimiento,area_sotano_habitable1,area_sotano_habitable2,...,area_garage,area_terraza_madera,area_porche_abierto,area_porche_cerrado,area_porche_3estaciones,area_porche_tapado,area_piscina,valor_atributo_miscelaneo,mes_venta,ano_venta
0,20,141.0,31770,6,5,1960,1960,112.0,639.0,0.0,...,528.0,210,62,0,0,0,0,0,5,2010
1,20,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,...,730.0,140,0,0,0,120,0,0,6,2010
2,20,81.0,14267,6,6,1958,1958,108.0,923.0,0.0,...,312.0,393,36,0,0,0,0,12500,6,2010
3,20,93.0,11160,7,5,1968,1968,0.0,1065.0,0.0,...,522.0,0,0,0,0,0,0,0,4,2010
4,60,74.0,13830,5,5,1997,1998,0.0,791.0,0.0,...,482.0,212,34,0,0,0,0,0,3,2010


In [13]:
datos_categoricos.head()

,atributo_miscelaneo,cercania_carretera2,cercania_carretera1,LandContour,tipo_edificio,tipo_garaje,config_parcela,barrio,material_exterior2,aire_acondicionado_central,...,tipo_revestimiento,tipo_casa,tipo_tejado,tipo_zona,tipo_acceso_trasero,material_exterior1,tipo_acceso,sistema_electrico,tipo_calefaccion,tipo_venta
0,NaN,Norm,Norm,Lvl,1Fam,Attchd,Corner,NAmes,Plywood,Y,...,Stone,1Story,Hip,RL,NaN,BrkFace,Pave,SBrkr,GasA,WD
1,NaN,Norm,Feedr,Lvl,1Fam,Attchd,Inside,NAmes,VinylSd,Y,...,None,1Story,Gable,RH,NaN,VinylSd,Pave,SBrkr,GasA,WD
2,Gar2,Norm,Norm,Lvl,1Fam,Attchd,Corner,NAmes,Wd Sdng,Y,...,BrkFace,1Story,Hip,RL,NaN,Wd Sdng,Pave,SBrkr,GasA,WD
3,NaN,Norm,Norm,Lvl,1Fam,Attchd,Corner,NAmes,BrkFace,Y,...,None,1Story,Hip,RL,NaN,BrkFace,Pave,SBrkr,GasA,WD
4,NaN,Norm,Norm,Lvl,1Fam,Attchd,Inside,Gilbert,VinylSd,Y,...,None,2Story,Gable,RL,NaN,VinylSd,Pave,SBrkr,GasA,WD


In [14]:
datos_ordinales.head()

,calidad_cocinas,funcionalidad,calidad_chimeneas,acabado_garaje,calidad_garaje,condicion_garaje,acceso_garaje_pavimentado,calidad_piscina,calidad_valla,forma_parcela,tipo_instalaciones,pendiente_parcela,calidad_material_exterior,condicion_material_exterior,altura_sotano,condicion_sotano,sotano_exterior,calidad_sotano_habitable1,calidad_sotano_habitable2,calidad_calefaccion
0,TA,Typ,Gd,Fin,TA,TA,P,NaN,NaN,IR1,AllPub,Gtl,TA,TA,TA,Gd,Gd,BLQ,Unf,Fa
1,TA,Typ,NaN,Unf,TA,TA,Y,NaN,MnPrv,Reg,AllPub,Gtl,TA,TA,TA,TA,No,Rec,LwQ,TA
2,Gd,Typ,NaN,Unf,TA,TA,Y,NaN,NaN,IR1,AllPub,Gtl,TA,TA,TA,TA,No,ALQ,Unf,TA
3,Ex,Typ,TA,Fin,TA,TA,Y,NaN,NaN,Reg,AllPub,Gtl,Gd,TA,TA,TA,No,ALQ,Unf,Ex
4,TA,Typ,TA,Fin,TA,TA,Y,NaN,MnPrv,IR1,AllPub,Gtl,TA,TA,Gd,TA,No,GLQ,Unf,Gd


Nos aseguramos que todas las columnas están clasificadas como categóricas o numéricas

In [15]:
[col for col in ames.columns if col not in datos_numericos.columns and\
 col not in datos_categoricos.columns and \
 col not in datos_ordinales.columns]

['precio_venta']

## Variables numéricas

con las variables numéricas muy sencillo, vamos a eliminar valores inexistentes (reemplazandolos pr normalizarlas

In [16]:
from sklearn.preprocessing import Imputer, normalize


# Recordatorio: los transformadores de sklearn devuelven numpy arrays, no dataframes
datos_numericos_imputados_normalizados = pd.DataFrame(
    normalize(Imputer(strategy="median").fit_transform(datos_numericos)),
    columns=datos_numericos.columns
)

### Variables categóricas

Con las variables categóricas tenemos dos opciones principalmente:
- Usar `sklearn.preprocessing.LabelBinarizer` para hacer un 1 hot encoding y codificarlas como vectores
- Usar la función get_dummies de pandas.

En general la opción recomendada es la de usar `LabelBinarizer` ya que esto nos crea un transformador de scikit learn que podemos usar en pipelines y para transformar nuevas observaciones. Sin embargo dado que esta sección es sobre selección de variables y para no hacerla demasiado larga, directamente voy a usar `pd.get_dummies`.

In [17]:
datos_categoricos_dummy = pd.get_dummies(datos_categoricos, drop_first=True)

In [18]:
datos_categoricos_dummy.shape

(2930, 153)

### Variables ordinales

Yendo al [diccionario de datos](https://ww2.amstat.org/publications/jse/v19n3/decock/DataDocumentation.txt) (donde se describen todas las variables) hay varias variables que son ordinales (Ordinal), básicamente evaluaciones de una condición de la casa definidas en una escala no numérica pero que tiene cierto orden (de "mejor" a "peor").

Scikit-learn no tiene un Codificador ordinal (hasta ahora hemos usado el label encoder.) Tenemos tres opciones:

1. Implementar nuestro propio `OrdinalEncoder`.
2. Utilizar la implementación de `OrdinalEncoder` del paquete [scikit-learn.contrib.categorical_encoding](https://github.com/scikit-learn-contrib/categorical-encoding), que podemos instalar con 

```conda install -c conda-forge category_encoders
ó
pip install category_encoders
```
3. Ya que estamos usando pandas, podemos convertir las variables ordinales a categorías de pandas, especificar sus niveles y convertirlas a su valor numérico. Vamos a usar esta opción.

In [ ]:
dict_var_ordinales = {
    "calidad_cocinas": ["Po", "Fa", "TA", "Gd", "Ex"],
    "funcionalidad":["Sal", "Sev", "Maj2", "Maj1", "Min2", "Min1","Typ"],
    "calidad_chimeneas":["NA","Po","Fa","TA","Gd","Ex"],
    "acabado_garaje":["NA","Unf","RFn","Fin"],
    "calidad_garaje":["NA","Po","Fa","TA","Gd","Ex"],
    "condicion_garaje":["NA","Po","Fa","TA","Gd","Ex"],
    "acceso_garaje_pavimentado":["N", "P", "Y"],
    "calidad_piscina":["NA","Fa","TA","Gd","Ex"],
    "calidad_valla":["NA","MnWw","GdWo","MnPrv","GdPrv"],
    "forma_parcela":["IR3", "IR2", "IR1","Reg"],
    "tipo_instalaciones":["ELO","NoSeWa","NoSewr","AllPub"],
    "pendiente_parcela":["Sev", "Mod", "Gtl"],
    "calidad_material_exterior":["Po","Fa","TA","Gd","Ex"],
    "condicion_material_exterior":["Po","Fa","TA","Gd","Ex"],
    "altura_sotano":["NA","Po","Fa","TA","Gd","Ex"],
    "condicion_sotano":["NA","Po","Fa","TA","Gd","Ex"],
    "sotano_exterior":["NA","No","Mn","Av","Gd"],
    "calidad_sotano_habitable1":["NA", "Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"],
    "calidad_sotano_habitable2":["NA", "Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"],
    "calidad_calefaccion":["Po","Fa","TA","Gd","Ex"],
}

In [19]:
for columna_ordinal, valores in dict_var_ordinales.items():
    datos_ordinales[columna_ordinal] = (
    datos_ordinales[columna_ordinal]
    .astype("category")
    .cat.set_categories(valores)
    .cat.codes
)

/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [20]:
datos_ordinales.head()

,calidad_cocinas,funcionalidad,calidad_chimeneas,acabado_garaje,calidad_garaje,condicion_garaje,acceso_garaje_pavimentado,calidad_piscina,calidad_valla,forma_parcela,tipo_instalaciones,pendiente_parcela,calidad_material_exterior,condicion_material_exterior,altura_sotano,condicion_sotano,sotano_exterior,calidad_sotano_habitable1,calidad_sotano_habitable2,calidad_calefaccion
0,2,6,4,3,3,3,1,-1,-1,2,3,2,2,2,3,4,4,4,1,1
1,2,6,-1,1,3,3,2,-1,3,3,3,2,2,2,3,3,1,3,2,2
2,3,6,-1,1,3,3,2,-1,-1,2,3,2,2,2,3,3,1,5,1,2
3,4,6,3,3,3,3,2,-1,-1,3,3,2,3,2,3,3,1,5,1,4
4,2,6,3,3,3,3,2,-1,3,2,3,2,2,2,4,3,1,6,1,3


In [22]:
### Unimos los tres tipos de datos

ames_procesado = pd.concat([
    datos_numericos_imputados_normalizados,
    datos_categoricos_dummy,
    datos_ordinales
], axis=1)

In [23]:
ames_procesado.shape

(2930, 209)

Ya tenemos un dataset preparado para poder entrenar modelos. Este dataset tiene una complejidad dimensional alta ( 209 variables independientes), por lo que vamos a usar técnicas de selección de variables para reducirla.

Antes que nada vamos a ver que errores obtenemos con diversos modelos entrenando con el dataset con todas las variables.

In [25]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression


ames_X = ames_procesado
ames_y = ames[variable_objetivo]

Vamos a usar la función `cross_validate` que es una versión más flexible que `cross_val_score`. Evaluaremos usando la raíz del error cuadrático medio (RMSE)

In [26]:
def rmse(y_real, y_pred):
    return np.sqrt(mean_squared_error(y_real, y_pred))

def rmse_cv(estimator, X, y):
    y_pred = estimator.predict(X)
    return rmse(y, y_pred)

res = cross_validate(LinearRegression(), ames_procesado, ames[variable_objetivo],
                     scoring=rmse_cv, n_jobs=-1, cv=10)

Dicha función devuelve no solo las puntuaciones en los datos de testing, sino también en los datos de entrenamiento. Además, nos devuelve el tiempo que tarda dicho estimador en ajustar el modelo y en hacer predicciones con el mismo.

In [28]:
res

/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


{'fit_time': array([ 0.20597267,  0.22455931,  0.209306  ,  0.51587296,  0.66813993,
         0.53761435,  0.40231967,  0.37008762,  0.34896564,  0.35558343]),
 'score_time': array([ 0.00364852,  0.00419474,  0.00626087,  0.01019692,  0.0048418 ,
         0.00396061,  0.01076198,  0.00665569,  0.00999808,  0.0049293 ]),
 'test_score': array([  3.06953499e+04,   3.64285081e+04,   2.50088517e+04,
          3.02256878e+04,   6.46029890e+13,   3.75839899e+04,
          3.81302496e+04,   3.60406418e+04,   3.14541175e+04,
          3.09467457e+04]),
 'train_score': array([ 27000.04190286,  26303.34836791,  27653.52874198,  27084.18421673,
         27681.61303207,  26791.51076946,  26600.19518926,  26580.10313152,
         26869.87702619,  27521.10920957])}

In [29]:
def evaluar_modelo(estimador, X, y):
    resultados_estimador = cross_validate(estimador, X, y,
                     scoring=rmse_cv, n_jobs=-1, cv=10, return_train_score=True)
    return resultados_estimador

In [30]:
resultados = {}

def ver_resultados():
    resultados_df  = pd.DataFrame(resultados).T
    resultados_cols = resultados_df.columns
    for col in resultados_df:
        resultados_df[col] = resultados_df[col].apply(np.mean)
        resultados_df[col+"_idx"] = resultados_df[col] / resultados_df[col].min()
    return resultados_df

In [31]:
resultados["reg_lineal_sin_seleccion"] = evaluar_modelo(LinearRegression(), ames_X, ames_y)
resultados["svr_sin_seleccion"] = evaluar_modelo(SVR(), ames_X, ames_y)
resultados["rf_sin_seleccion"] = evaluar_modelo(RandomForestRegressor(), ames_X, ames_y)

/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when 

In [32]:
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
reg_lineal_sin_seleccion,0.411740,0.012721,6.460299e+12,27008.551159,1.000000,3.631005,1.969080e+08,1.921155
rf_sin_seleccion,1.532122,0.003503,3.280871e+04,14058.500059,3.721093,1.000000,1.000000e+00,1.000000
svr_sin_seleccion,5.891846,0.536793,8.104416e+04,82161.012024,14.309633,153.218660,2.470203e+00,5.844223


Hay 3 tipos generales de estrategias de selección de variables:

# Métodos de filtrado

Los métodos de filtrado usan métodos estadísticos para seleccionar las variables que proporcionan la mayor cantidad de información. Estos métodos se aplican de forma previa a entrenar el modelo (preprocesado), y **son completamente independientes de la elección del estimador**. Generalmente funcionan definiendo una función de evaluación $S(xk_i, y_k)$, evaluando cada variable independiente para cada observación respecto a la variable objetivo de dicha observación, y eligiendo aquellas `K` variables que mejor funcionan.



Scikit-learn tiene las siguientes funciones de evaluación:

- Para regresión: `f_regression, mutual_info_regression`
- Para clasificación: `chi2, f_classif, mutual_info_classif`

f_regression y f_classif devuelven [estadísticos F](https://es.wikipedia.org/wiki/Estad%C3%ADstico_F) (F-values), entrenando un modelo lineal entre las variables independientes y la objetivo en el caso de regresión, y un test ANOVA en el caso de clasificación.

`mutual_info_regression` y `mutual_info_classif` computan el coeficiente de información mutua (MIC) entre las variables independientes y la variable objetivo.

El coeficiente de información mutua nos da una medida de la dependencia entre las variables. El MIC entre dos variables es 0 si no hay relación entre las mismas, y aumenta conforme más relación tienen.

El MIC se define como:

$$I(X;Y) = \sum_{x,y} P_{XY}(x,y) \log
{P_{XY}(x,y) \over P_X(x) P_Y(y)}$$

con $P_{XY}$ siendo la probabilidad conjunta de X e Y


El evaluador `chi2` calcula el estadístico chi cuadrado y lo convierte a estadísticos F, con el estadístico chi cuadrado definido como:
  $$\chi^2(x,y) = \sum {\frac{(x-y)^2}{y}}$$


In [33]:
from sklearn.feature_selection import SelectKBest, f_regression

In [34]:
selector_kbest10 = SelectKBest(f_regression, k=10)
ames_X_kbest10 = selector_kbest10.fit_transform(ames_X, ames_y)

/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [35]:
ames_X_kbest10.shape

(2930, 10)

La funcion `get_support` nos devuelve un vector booleano (True/False), aquellos elementos con True son las columnas que se han seleccionado

In [37]:
columnas_seleccion_kbest10 = ames_X.loc[:,selector_kbest10.get_support()].columns
columnas_seleccion_kbest10

Index(['barrio_NridgHt', 'tipo_andamios_PConc', 'tipo_revestimiento_None',
       'calidad_cocinas', 'calidad_chimeneas', 'acabado_garaje',
       'calidad_material_exterior', 'altura_sotano', 'sotano_exterior',
       'calidad_calefaccion'],
      dtype='object')

El parámetro `scores_` del selector nos devuelve los resultados de la función de evaluación

In [38]:
selector_kbest10.scores_[:10]

array([  98.54044172,   46.4426689 ,  140.45170476,   11.38580355,
        507.38479194,  470.33287137,  495.34300746,  320.65956248,
        215.58416354,    3.89847629])

In [39]:
puntuaciones_seleccion_kbest10 = zip(ames_X.columns, selector_kbest10.scores_,
                                     selector_kbest10.get_support())
evaluacion_kbest10 = sorted(
                filter(lambda c: c[2], puntuaciones_seleccion_kbest10),
                key=lambda c: c[1], reverse=True
)

In [40]:
list(evaluacion_kbest10)

[('calidad_material_exterior', 2781.4056293390681, True),
 ('calidad_cocinas', 2423.0002986124264, True),
 ('altura_sotano', 1372.4642743061177, True),
 ('acabado_garaje', 1163.345116865557, True),
 ('calidad_chimeneas', 1124.6033740020016, True),
 ('tipo_andamios_PConc', 1090.6972231858886, True),
 ('calidad_calefaccion', 744.81949029726775, True),
 ('barrio_NridgHt', 676.79274534914623, True),
 ('sotano_exterior', 604.35564563926209, True),
 ('tipo_revestimiento_None', 570.98218768093648, True)]

Esto nos permite ver cual es la puntuación que le da el evaluador `f_regression` a cada variable independiente

In [41]:
resultados["reg_lineal_kbest_10"] = evaluar_modelo(LinearRegression(), ames_X_kbest10, ames_y)
resultados["rf_kbest_10"] = evaluar_modelo(RandomForestRegressor(), ames_X_kbest10, ames_y)
resultados["svr_kbest_10"] = evaluar_modelo(SVR(), ames_X_kbest10, ames_y)

/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/model_s

In [42]:
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
reg_lineal_kbest_10,0.037006,0.003586,4.474449e+04,44962.297898,1.000000,1.568803,1.363799e+00,3.198229
reg_lineal_sin_seleccion,0.411740,0.012721,6.460299e+12,27008.551159,11.126243,5.565142,1.969080e+08,1.921155
rf_kbest_10,0.078170,0.002286,4.632880e+04,31883.442706,2.112359,1.000000,1.412088e+00,2.267912
rf_sin_seleccion,1.532122,0.003503,3.280871e+04,14058.500059,41.401786,1.532673,1.000000e+00,1.000000
svr_kbest_10,0.808120,0.048932,8.090850e+04,82043.551184,21.837426,21.406415,2.466068e+00,5.835868
svr_sin_seleccion,5.891846,0.536793,8.104416e+04,82161.012024,159.212464,234.834065,2.470203e+00,5.844223


Vemos que al reducir las dimensiones a solo 10, los modelos lineales y RF funcionan bastante peor, aunque vemos que por otra parte las diferencias entre la evaluación de entrenamiento y test se han reducido en todos los casos. Esto es así por que al tener menos variables, hay mucha menos flexibilidad para sobreajustar (es decir, "memorizar" los datos de entrenamiento). SVM sin embargo funciona más o menos igual (de mal), pero los tiempos de entrenamiento se reducen considerablemente (de 3.9 segundos a 0.7).

Probamos ahora para 50 variables

In [44]:
selector_kbest50 = SelectKBest(f_regression, k=50)
ames_X_kbest50 = selector_kbest50.fit_transform(ames_X, ames_y)

/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [45]:
resultados["reg_lineal_kbest_50"] = evaluar_modelo(LinearRegression(), ames_X_kbest50, ames_y)
resultados["rf_kbest_50"] = evaluar_modelo(RandomForestRegressor(), ames_X_kbest50, ames_y)
resultados["svr_kbest_50"] = evaluar_modelo(SVR(), ames_X_kbest50, ames_y)

/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/model_s

In [46]:
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
reg_lineal_kbest_10,0.037006,0.003586,4.474449e+04,44962.297898,1.000000,1.568803,1.363799e+00,3.198229
reg_lineal_kbest_50,0.083352,0.013197,3.473844e+04,33794.489434,2.252375,5.773194,1.058818e+00,2.403847
reg_lineal_sin_seleccion,0.411740,0.012721,6.460299e+12,27008.551159,11.126243,5.565142,1.969080e+08,1.921155
rf_kbest_10,0.078170,0.002286,4.632880e+04,31883.442706,2.112359,1.000000,1.412088e+00,2.267912
rf_kbest_50,0.555823,0.002432,3.493510e+04,14532.714954,15.019731,1.063729,1.064812e+00,1.033732
rf_sin_seleccion,1.532122,0.003503,3.280871e+04,14058.500059,41.401786,1.532673,1.000000e+00,1.000000
svr_kbest_10,0.808120,0.048932,8.090850e+04,82043.551184,21.837426,21.406415,2.466068e+00,5.835868
svr_kbest_50,1.768061,0.115177,8.097104e+04,82091.449480,47.777449,50.387213,2.467974e+00,5.839275
svr_sin_seleccion,5.891846,0.536793,8.104416e+04,82161.012024,159.212464,234.834065,2.470203e+00,5.844223


Vemos que para 50 variables, los tres estimadores funcionan de forma similar al modelo entrenado con las 209 variables iniciales, sin embargo los tiempos de entrenamiento de los mismos se reducen considerablemente.

De forma similar a `SelectKBest`, donde le decimos al transformador el número de variables que queremos mantener, podemos usar [`SelectPercentile`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectPercentile.html#sklearn.feature_selection.SelectPercentile) donde especificamos el porcentaje de variables sobre el total.

Por ejemplo, si solo quisieramos conservar el 10% de las variables (sin importar que tengamos 200 o 20000), podemos hacer lo siguiente:


In [47]:
from sklearn.feature_selection import SelectPercentile

selector_pct10 = SelectPercentile(f_regression, percentile=10)
ames_X_pct10 = selector_pct10.fit_transform(ames_X, ames_y)

/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [48]:
ames_X_pct10.shape

(2930, 21)

El transformador nos selecciona 21 variables (el 10 por ciento de 209 redondeado)

# Métodos envolventes (wrapper methods)

Los métodos envolventes *(wrapper methods)* funcionan de forma similar a los métodos de ranking. Sin embargo, en lugar de usar una función estadística independiente del modelo para evaluar las variables, estos métodos usan la función de evaluación o el performance de los modelos como input para decidir que variables elegir (es decir, "envuelven" el funcionamiento del estimador). Ésto significa que los métodos de filtrado se pueden aplicar independientemente de la elección del modelo, ya que consideran los modelos como una caja negra que produce evaluaciones, aunque claro, diferentes modelos producirán diferentes selecciones de variables.

`Scikit-learn` implementa un metodo envolvente llamado **Recursive Feature Elimination [(RFE)](http://scikit-learn.org/stable/modules/feature_selection.html#recursive-feature-elimination)**, o Eliminación Recursiva de Variables. RFE funciona seleccionando todas las variables, entrenando el modelo, usando los coeficientes `coef_` o la importancia de las variables `feature_importances_` en función del estimador, y eliminando n variables. Este proceso se repite hasta que se alcanza el número de variables deseado.

In [49]:
from sklearn.feature_selection import RFE
estimador_selector = RandomForestRegressor()
selector_rfe10_rf = RFE(estimador_selector, n_features_to_select=10)
ames_X_rfe10_rf = selector_rfe10_rf.fit_transform(ames_X, ames_y)

/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [50]:
ames_X_rfe10_rf.shape

(2930, 10)

In [51]:
evaluacion_rfe10_rf = sorted(
    filter(lambda c: c[2], 
        zip(
            ames_X.columns,
            selector_rfe10_rf.ranking_,
            selector_rfe10_rf.get_support()
        )
    ), key=lambda c: c[1],reverse=True
)

In [52]:
evaluacion_rfe10_rf

[('condicion_general', 1, True),
 ('area_sotano_habitable1', 1, True),
 ('area_habitable_sobre_suelo', 1, True),
 ('ano_construccion_garaje', 1, True),
 ('n_coches_garaje', 1, True),
 ('ano_venta', 1, True),
 ('calidad_cocinas', 1, True),
 ('calidad_chimeneas', 1, True),
 ('calidad_material_exterior', 1, True),
 ('altura_sotano', 1, True)]

In [54]:
resultados["reg_lineal_rfe10_rf"] = evaluar_modelo(LinearRegression(), ames_X_rfe10_rf, ames_y)
resultados["rf_rfe10_rf"] = evaluar_modelo(RandomForestRegressor(), ames_X_rfe10_rf, ames_y)
resultados["svr_rfe10_rf"] = evaluar_modelo(SVR(), ames_X_rfe10_rf, ames_y)

/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/model_s

Si usamos otro estimador para evaluar veremos que las variables elegidas pueden ser completamente distintas. Los estimadores que se pueden usar tienen que implementar el metodo `coef_` o el metodo `feature_importance` (es decir, tienen que tener una manera de ordenar variables en función de su importancia). Por ejemplo, no podemos usar SVMs.

In [55]:
estimador_selector = LinearRegression()
selector_rfe10_lineal = RFE(estimador_selector, n_features_to_select=10)
ames_X_rfe10_lineal = selector_rfe10_lineal.fit_transform(ames_X, ames_y)

/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [57]:
evaluacion_rfe10_lineal = sorted(
    filter(lambda c: c[2], 
        zip(
            ames_X.columns,
            selector_rfe10_lineal.ranking_,
            selector_rfe10_lineal.get_support()
        )
    ), key=lambda c: c[1],reverse=True
)
evaluacion_rfe10_lineal

[('calidad_general', 1, True),
 ('condicion_general', 1, True),
 ('n_aseos_sotanos', 1, True),
 ('n_medios_aseos_sotanos', 1, True),
 ('n_aseos_sobre_suelo', 1, True),
 ('n_dormitorios_sobre_suelo', 1, True),
 ('n_cocinas', 1, True),
 ('n_habitaciones_sobre_suelo', 1, True),
 ('n_chimeneas', 1, True),
 ('n_coches_garaje', 1, True)]

In [60]:
set(evaluacion_rfe10_rf).intersection(set(evaluacion_rfe10_lineal))

{('condicion_general', 1, True), ('n_coches_garaje', 1, True)}

In [61]:
resultados["reg_lineal_rfe10_lineal"] = evaluar_modelo(LinearRegression(), ames_X_rfe10_lineal, ames_y)
resultados["rf_rfe10_lineal"] = evaluar_modelo(RandomForestRegressor(), ames_X_rfe10_lineal, ames_y)
resultados["svr_rfe10_lineal"] = evaluar_modelo(SVR(), ames_X_rfe10_lineal, ames_y)

/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/model_s

In [62]:
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
reg_lineal_kbest_10,0.037006,0.003586,4.474449e+04,44962.297898,1.162318,7.310635,1.370318e+00,3.198229
reg_lineal_kbest_50,0.083352,0.013197,3.473844e+04,33794.489434,2.617975,26.903130,1.063878e+00,2.403847
reg_lineal_rfe10_lineal,0.033326,0.000491,5.595427e+04,55527.439842,1.046739,1.000000,1.713621e+00,3.949741
reg_lineal_rfe10_rf,0.031838,0.003445,3.787494e+04,37889.595060,1.000000,7.023671,1.159935e+00,2.695138
reg_lineal_sin_seleccion,0.411740,0.012721,6.460299e+12,27008.551159,12.932229,25.933606,1.978492e+08,1.921155
rf_kbest_10,0.078170,0.002286,4.632880e+04,31883.442706,2.455232,4.660008,1.418838e+00,2.267912
rf_kbest_50,0.555823,0.002432,3.493510e+04,14532.714954,17.457699,4.956985,1.069901e+00,1.033732
rf_rfe10_lineal,0.254774,0.002176,3.857513e+04,16273.402142,8.002117,4.437056,1.181379e+00,1.157549
rf_rfe10_rf,0.221563,0.002548,3.265264e+04,14063.125492,6.959001,5.195101,1.000000e+00,1.000329
rf_sin_seleccion,1.532122,0.003503,3.280871e+04,14058.500059,48.122026,7.142267,1.004780e+00,1.000000


Vemos que el estimador RF entrenado con solo 10 variables seleccionadas por `RFE` basado en RF es el modelo que mejor funciona, y tarda 8 veces menos en entrenar.

scikit-learn también tiene la implementación [RFECV](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFECV.html#sklearn.feature_selection.RFECV) que implementa el mismo algoritmo pero hace validación cruzada y entrena en múltiples particiones del dataset para calcular automáticamente el conjunto de variables que funciona mejor (a costa de un mayor tiempo de procesamiento).

In [63]:
from sklearn.feature_selection import RFECV

estimador_selector = RandomForestRegressor()
selector_rfecv = RFECV(estimador_selector)
ames_X_rfecv = selector_rfecv.fit_transform(ames_X, ames_y)

/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [64]:
ames_X_rfecv.shape

(2930, 188)

En este caso `RFECV` ha elegido 188 variables como aquellas que proporcionan el mejor funcionamiento al estimador RF utilizado.

In [65]:
resultados["reg_lineal_rfecv_rf"] = evaluar_modelo(LinearRegression(), ames_X_rfecv, ames_y)
resultados["rf_rfecv_rf"] = evaluar_modelo(RandomForestRegressor(), ames_X_rfecv, ames_y)
resultados["svr_rfecv_rf"] = evaluar_modelo(SVR(), ames_X_rfecv, ames_y)

/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/manuel/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/model_s

In [66]:
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
reg_lineal_kbest_10,0.037006,0.003586,4.474449e+04,44962.297898,1.162318,7.310635,1.370318e+00,3.198229
reg_lineal_kbest_50,0.083352,0.013197,3.473844e+04,33794.489434,2.617975,26.903130,1.063878e+00,2.403847
reg_lineal_rfe10_lineal,0.033326,0.000491,5.595427e+04,55527.439842,1.046739,1.000000,1.713621e+00,3.949741
reg_lineal_rfe10_rf,0.031838,0.003445,3.787494e+04,37889.595060,1.000000,7.023671,1.159935e+00,2.695138
reg_lineal_rfecv_rf,0.454398,0.003780,1.451599e+14,27671.262437,14.272053,7.705210,4.445580e+09,1.968294
reg_lineal_sin_seleccion,0.411740,0.012721,6.460299e+12,27008.551159,12.932229,25.933606,1.978492e+08,1.921155
rf_kbest_10,0.078170,0.002286,4.632880e+04,31883.442706,2.455232,4.660008,1.418838e+00,2.267912
rf_kbest_50,0.555823,0.002432,3.493510e+04,14532.714954,17.457699,4.956985,1.069901e+00,1.033732
rf_rfe10_lineal,0.254774,0.002176,3.857513e+04,16273.402142,8.002117,4.437056,1.181379e+00,1.157549
rf_rfe10_rf,0.221563,0.002548,3.265264e+04,14063.125492,6.959001,5.195101,1.000000e+00,1.000329


Vemos que el método que funciona mejor es el de RFECV usando RF como estimador. Este método funciona mejor que  el segundo mejor modelo, un Random Forest entrenado con todo el dataset, pero se entrena en menos tiempo (0.81 segundos versus  0.99 segundos)

# Metodos embebidos (Embedded Methods)

Éstos métodos son metodos de selección de variables que están integrados dentro del proceso de entrenamiento de modelos, y son específicos para cada modelo.

El ejemplo clásico de método embebidos de selección de variables son los procesos de regularización en regresión lineal. Por ejemplo el método L1 tiende a convertir los coeficientes de las variables que no funcionan a 0, por lo tanto eliminandolas.